In [4]:
%load_ext autoreload
%autoreload 2

import glob
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from PIL import Image
from pathlib import Path


import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
# from sklearn.utils.class_weight import compute_class_weight

from sklearn.utils import class_weight

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from torchvision.models  import resnet101, ResNet101_Weights
from torchvision import models
from torchvision.io import read_image
from torchvision.transforms import Resize
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
from tqdm import tqdm
from pathlib import Path
import os
# from torchsummary import summary
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PyTorch Version:  2.0.0
Torchvision Version:  0.15.0


In [5]:
np.random.seed(31101995)
torch.manual_seed(31101995)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
# Set seed
seed_everything(31101995)

In [ ]:
meta_csv_path = '/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/meta_data_survival_mapping.csv'

class_column_name = 'label_flair'
subject_id_column_name = 'subject_id'

healthy = 'healthy'
discard = 'discard'
test_size = 0.3

In [ ]:
df_meta = pd.read_csv(meta_csv_path)
del df_meta['Brats20ID']
del df_meta['Age']
del df_meta['Survival_days']
del df_meta['Extent_of_Resection']
df_meta

def remove_columns(meta_csv_path):    
    df_meta = pd.read_csv(meta_csv_path)
    del df_meta['Brats20ID']
    del df_meta['Age']
    del df_meta['Survival_days']
    del df_meta['Extent_of_Resection']
    return df_meta
    

In [ ]:
# max(df_meta.groupby(['subject_id'])['label_flair'].count())

In [ ]:
def get_class_label_dict(df, class_column):
    class_dict = df[class_column].value_counts().to_dict()
    return class_dict

class_dict = get_class_label_dict(df_meta, class_column_name)
class_dict

In [ ]:
def get_minority_class_values(df_meta, class_dict, 
                              subject_id_column_name, 
                              class_column_name):
    
    samples_per_subject = max(df_meta.groupby([subject_id_column_name])[class_column_name].count())
    print('samples_per_subject:', samples_per_subject)
    
    minority_samples = min(class_dict.values())
    minority_class = [k for k, v in class_dict.items() if v == minority_samples]
    minority_class = minority_class[0]
    print('minority_class:',minority_class)

    return minority_class, samples_per_subject

In [ ]:
samples_per_subject = max(df_meta.groupby([subject_id_column_name])[class_column_name].count())#155
print('samples_per_subject:', samples_per_subject)
minority_samples = min(class_dict.values())
minority_class = [k for k, v in class_dict.items() if v == minority_samples]
minority_class = minority_class[0]
print('minority_class:',minority_class)

# df = df_meta.copy()

healthy = 'healthy'
discard = 'discard'
test_size = 0.3

In [ ]:
#STEPS:
#1.Get max number od samples/images per subject (=155)
#2.Find the minority class_label
#3.Set test_size : 
    #a. 30% minority class
    #b. get number of samples in 30% of minority class
    #c. get same number of samples from all other classes
    # therefore, 30% of minority_class_samples = number of samples per class in test_set

In [ ]:
def get_minority_class_train_test_split(df_meta, class_dict,
                                  minority_class,test_size,
                                  subject_id_column_name):
    
    df = df_meta.copy()
    for key in class_dict:
        if key == minority_class:
            df_class =  df[df[class_column_name] == key]
            subject_id_list =  np.unique(df_class[subject_id_column_name]).tolist()
            
            train_subjects_minority, test_subjects_minority = train_test_split(subject_id_list ,
                                            test_size = test_size, 
                                            random_state = 42, 
                                            shuffle = True)
            
    print(len(train_subjects_minority), len(test_subjects_minority))
    return train_subjects_minority, test_subjects_minority
    

In [ ]:
df = df_meta.copy()

for key in class_dict:
    if key==minority_class:
#         print(key)
        df_class =  df[df[class_column_name] == key]
        subject_id_list =  np.unique(df_class[subject_id_column_name]).tolist()
        
        train_subjects_minority, test_subjects_minority = train_test_split(subject_id_list ,
                                            test_size = test_size, 
                                            random_state = 42, 
                                            shuffle = True)
        
        
df_class['subject_id'].value_counts()
print(len(subject_id_list))

In [ ]:
len(train_subjects_minority), len(test_subjects_minority)

In [ ]:
def get_majority_class_train_test_split(df_meta, class_dict,
                                        minority_class, healthy,
                                        discard, class_column_name,
                                        subject_id_column_name,
                                        test_subjects_minority):
    
    train_subjects_other = []
    test_subjects_other = []
    df = df_meta.copy()
    
    for key in class_dict:
        if key!=healthy and key!=discard and key!=minority_class:            
            df_class_majority =  df[df[class_column_name] == key]
            subject_id_class =  np.unique(df_class_majority[subject_id_column_name]).tolist()            
            test_subjects = random.sample(subject_id_class, len(test_subjects_minority))            
            train_subjects = list(set(subject_id_class) - set(test_subjects))            
            train_subjects_other.append(train_subjects)
            test_subjects_other.append(test_subjects)
    
    return train_subjects_other, test_subjects_other

In [ ]:
train_subjects_other = []
test_subjects_other = []

for key in class_dict:
    if key!=healthy and key!=discard and key!=minority_class:
        print(key)
        df_class_majority =  df[df[class_column_name] == key]
        subject_id_class =  np.unique(df_class_majority[subject_id_column_name]).tolist()
#         print(random.sample(subject_id_class, len(test_subjects_minority)))
        test_subjects = random.sample(subject_id_class, len(test_subjects_minority))
    
        train_subjects = list(set(subject_id_class) - set(test_subjects))
        
        train_subjects_other.extend(train_subjects)
        test_subjects_other.extend(test_subjects)
        
        
#         print(test_subjects)
#         print('train:',train_subjects)
        print(len(test_subjects))
        print(len(subject_id_class))

In [ ]:
def create_save_train_test_split(train_subjects_other, test_subjects_other,
                                train_subjects_minority, test_subjects_minority):
    
    df = get_csv_after_discard(meta_csv_path)
    training_subjects_list, test_subjects_list = generate_subject_id_splits(df,
                                                                          class_column, 
                                                                          test_size, 
                                                                          subject_id_column_name, 
                                                                          class_healthy_name)
    train_df = df[df[subject_id_column_name].isin(training_subjects_list)].reset_index(drop=True)
    test_df = df[df[subject_id_column_name].isin(test_subjects_list)].reset_index(drop=True)
    
    train_df.to_csv(csv_save_path / 'train_df.csv')
    test_df.to_csv(csv_save_path / 'test_df.csv')
    return train_df, test_df

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold

class_column_name = 'label_flair'
subject_id_column_name = 'subject_id'

healthy = 'healthy'
discard = 'discard'
test_size = 0.3

def class_to_idx(label):
    idx = 0
    if label == 'discard':idx =4
    if label == 'healthy':idx =0
    if label == 'HGG':idx =1
    if label=='LGG': idx = 2        
    return idx

def idx_to_class(idx):
    
    if idx == 4:label='discard'
    if idx == 0:label='healthy'
    if idx == 1:label='HGG'
    if idx == 2:label='LGG'      
    return label


def get_metadata_csv_label_encoded(meta_data_df):
    """Load meta csv. Filter samples to be discarded and label encoding."""
    meta_data = meta_data_df.copy()
    meta_data['label_flair'] = meta_data['label_flair'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1'] = meta_data['label_t1'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1ce'] = meta_data['label_t1ce'].apply(lambda x: class_to_idx(x))
    meta_data['label_t2'] = meta_data['label_t2'].apply(lambda x: class_to_idx(x))
    meta_data_df  = meta_data.reset_index(drop =True)
    return meta_data_df


def create_StratifiedGroupKFold_splits(k, train_df, subject_id_column_name): 
    
    train_df_encoded = get_metadata_csv_label_encoded(train_df)
    train_df_encoded['patient_id'] = train_df_encoded[subject_id_column_name].str[-3:].astype(int)
    
    kfold_dict = {f"train_{i}": [] for i in range(k)}
    kfold_dict.update({f"val_{i}": [] for i in range(k)})
    
    kfold_dist_dict = {f"train_{i}": [] for i in range(k)}
    kfold_dist_dict.update({f"val_{i}": [] for i in range(k)})

    n_splits = k
    # Initialize StratifiedGroupKFold
    sgkf = StratifiedGroupKFold(n_splits=n_splits)
    # Extract features and target
    X = train_df_encoded
    y = train_df_encoded['label_flair']
    # Define the groups (subject_id in your case)
    groups = train_df_encoded['patient_id']
    
    
    # Iterate through the splits
    for i, (train_index, val_index) in enumerate(sgkf.split(X, y,groups)):
        print(f"Fold {i}:")
#     for train_idx, val_idx in sgkf.split(X, y, groups=groups):
        # Ensure that the validation set is balanced
        unique_classes, class_counts = np.unique(y[val_idx], return_counts=True)
        min_class_count = min(class_counts)

        balanced_val_indices = []
        for cls in unique_classes:
            cls_indices = val_idx[y[val_idx] == cls]
            balanced_val_indices.extend(cls_indices[:min_class_count])

#         train_indices.append(train_idx)
#         val_indices.append(balanced_val_indices)
#         kfold_dict[f'train_{i}'].extend(groups[train_index])
        kfold_dict[f'val_{i}'].extend(balanced_val_indices)
        kfold_dict[f'train_{i}'].extend(train_index)

        kfold_dist_dict[f'val_{i}'] = dict(y[balanced_val_indices].value_counts())
        kfold_dist_dict[f'train_{i}'] = dict(y[train_index].value_counts())
        
#         train_dist =dict( y[train_index].value_counts())
#         val_dist =dict( y[balanced_val_indices].value_counts())
        # If you want to see the class distribution in each fold
        print(f'Fold {len(train_indices)} Class Distribution:')
        print(y[balanced_val_indices].value_counts())
    return kfold_dict, kfold_dist_dict
        

In [ ]:
# test_df = pd.read_csv('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/train_test_splits/test_df.csv',index_col =0)
# test_df[test_df['subject_id']== 'BraTS20_Training_212']

In [10]:
df = train_df[train_df['label_flair']!= 'discard'].reset_index(drop=True)
df

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,healthy
1,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
2,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
3,BraTS20_Training_001,BraTS20_Training_001_seg_102,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
4,BraTS20_Training_001,BraTS20_Training_001_seg_103,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
...,...,...,...,...,...,...,...
44588,BraTS20_Training_369,BraTS20_Training_369_seg_95,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
44589,BraTS20_Training_369,BraTS20_Training_369_seg_96,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
44590,BraTS20_Training_369,BraTS20_Training_369_seg_97,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
44591,BraTS20_Training_369,BraTS20_Training_369_seg_98,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG


In [6]:
test_df = pd.read_csv('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/train_test_splits/test_df.csv',index_col =0)
test_df = test_df[test_df['label_flair'] != 'discard']
# .reset_index(drop=True)
test_df

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_011,BraTS20_Training_011_seg_0,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,healthy
1,BraTS20_Training_011,BraTS20_Training_011_seg_1,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,healthy
2,BraTS20_Training_011,BraTS20_Training_011_seg_10,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,healthy
3,BraTS20_Training_011,BraTS20_Training_011_seg_100,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
4,BraTS20_Training_011,BraTS20_Training_011_seg_101,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
...,...,...,...,...,...,...,...
7125,BraTS20_Training_365,BraTS20_Training_365_seg_95,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
7126,BraTS20_Training_365,BraTS20_Training_365_seg_96,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
7127,BraTS20_Training_365,BraTS20_Training_365_seg_97,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
7128,BraTS20_Training_365,BraTS20_Training_365_seg_98,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG


In [14]:
len(test_df[test_df['label_flair']=='LGG']['subject_id'].value_counts())

23

In [3]:
train_df = pd.read_csv('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/train_test_splits/train_df.csv',index_col =0)
train_df

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/dataset/s...,discard,discard,discard,discard
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/dataset/s...,discard,discard,discard,discard
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,healthy
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
...,...,...,...,...,...,...,...
50060,BraTS20_Training_369,BraTS20_Training_369_seg_95,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
50061,BraTS20_Training_369,BraTS20_Training_369_seg_96,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
50062,BraTS20_Training_369,BraTS20_Training_369_seg_97,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG
50063,BraTS20_Training_369,BraTS20_Training_369_seg_98,/home/shsingh/knowledge_distillation/dataset/s...,HGG,HGG,HGG,HGG


In [32]:
df_class_weight.isnull().values.any()

False

In [7]:

df_class_weight = train_df[train_df['label_flair']!='discard']
df_class_weight = df_class_weight.fillna('healthy')

classes_list = df_class_weight['label_flair'].unique().tolist()
print(classes_list)
label_col =  df_class_weight['label_flair']

class_weights = compute_class_weight('balanced',classes =  classes_list, y = label_col)
class_weights
# class_weights = torch.tensor(class_weights, dtype=torch.float32)

['healthy', 'HGG', 'LGG']


array([0.64073164, 0.82428511, 4.42259248])

In [30]:
class_weights =compute_class_weight(class_weight = "balanced",
                        classes = df_class_weight['label_flair'].unique().tolist(),
                        y = df_class_weight['label_flair'])
class_weights

array([0.64073164, 0.82428511, 4.42259248])

In [31]:
class_weights = dict(zip(df_class_weight['label_flair'].unique().tolist(), class_weights))
class_weights

{'healthy': 0.6407316407316407,
 'HGG': 0.8242851069335847,
 'LGG': 4.422592482396112}

In [ ]:
# train_df = df[df[subject_id_column_name].isin(train_subjects_other)].reset_index(drop=True)
# train_df

In [4]:
train_df['label_flair'].value_counts()

healthy    23154
HGG        18033
discard     5472
LGG         3361
Name: label_flair, dtype: int64

In [5]:
def class_to_idx(label):
    idx = 0
    if label == 'discard':idx =4
    if label == 'healthy':idx =0
    if label == 'HGG':idx =1
    if label=='LGG': idx = 2        
    return idx


def get_metadata_csv_label_encoded(meta_data_df):
    """Load meta csv. Filter samples to be discarded and label encoding."""
    meta_data = meta_data_df.copy()
    meta_data['label_flair'] = meta_data['label_flair'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1'] = meta_data['label_t1'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1ce'] = meta_data['label_t1ce'].apply(lambda x: class_to_idx(x))
    meta_data['label_t2'] = meta_data['label_t2'].apply(lambda x: class_to_idx(x))
    meta_data_df  = meta_data.reset_index(drop =True)
    return meta_data_df



In [ ]:
train_df = get_metadata_csv_label_encoded(train_df)
train_df['patient_id'] = train_df['subject_id'].str[-3:].astype(int)
train_df

In [6]:
df1 = train_df[train_df.isna().any(axis=1)]
df1


,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
4699,BraTS20_Training_032,BraTS20_Training_032_seg_142,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,NaN
18652,BraTS20_Training_131,BraTS20_Training_131_seg_145,/home/shsingh/knowledge_distillation/dataset/s...,NaN,healthy,healthy,healthy
19176,BraTS20_Training_134,BraTS20_Training_134_seg_6,/home/shsingh/knowledge_distillation/dataset/s...,healthy,NaN,healthy,NaN
19187,BraTS20_Training_134,BraTS20_Training_134_seg_7,/home/shsingh/knowledge_distillation/dataset/s...,healthy,NaN,healthy,NaN
19220,BraTS20_Training_135,BraTS20_Training_135_seg_0,/home/shsingh/knowledge_distillation/dataset/s...,healthy,NaN,healthy,NaN
...,...,...,...,...,...,...,...
47785,BraTS20_Training_354,BraTS20_Training_354_seg_139,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,healthy,NaN
47944,BraTS20_Training_355,BraTS20_Training_355_seg_142,/home/shsingh/knowledge_distillation/dataset/s...,NaN,healthy,healthy,healthy
48254,BraTS20_Training_357,BraTS20_Training_357_seg_142,/home/shsingh/knowledge_distillation/dataset/s...,healthy,healthy,NaN,healthy
48593,BraTS20_Training_359,BraTS20_Training_359_seg_3,/home/shsingh/knowledge_distillation/dataset/s...,NaN,healthy,healthy,healthy


In [7]:
# train_df['label_flair']  = train_df['label_flair'].fillna('discard')
train_df= train_df.fillna('healthy')

In [ ]:
train_df.index

In [ ]:
# Find NaN values
nan_locations = train_df.isna()

# Count NaN values per column
nan_counts = train_df.isna().sum()

print("NaN Locations:")
print(nan_counts)

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold

# Assuming 'df' is your DataFrame and 'subject_id' is the column you want to use for grouping
# Assuming 'target' is the column representing the classes

# Define the number of splits
n_splits = 3

# Initialize StratifiedGroupKFold
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Extract features and target
# X = train_df['image_idx'] # Adjust 'target' with the actual column name
X = train_df
y = train_df['label_flair']

# Define the groups (subject_id in your case)
# groups = np.unique(train_df['subject_id'])
groups = train_df['patient_id']

In [ ]:
df = train_df.copy()

In [9]:
class_column_name = 'label_flair'
subject_id_column_name = 'subject_id'

healthy = 'healthy'
discard = 'discard'
test_size = 0.3
k = 3

In [10]:
from sklearn.model_selection import StratifiedGroupKFold

def create_StratifiedGroupKFold_splits(k, train_df,
                                        subject_id_column_name,
                                        class_column_name): 
    train_df= train_df.fillna('healthy')
    train_df_encoded = get_metadata_csv_label_encoded(train_df)
    train_df_encoded['patient_id'] = train_df_encoded[subject_id_column_name].str[-3:].astype(int)
    
    kfold_dict = {f"train_{i}": [] for i in range(k)}
    kfold_dict.update({f"val_{i}": [] for i in range(k)})
    
    kfold_dist_dict = {f"train_{i}": [] for i in range(k)}
    kfold_dist_dict.update({f"val_{i}": [] for i in range(k)})

    n_splits = k
    # Initialize StratifiedGroupKFold
    sgkf = StratifiedGroupKFold(n_splits=n_splits)
    # Extract features and target
    X = train_df_encoded
    y = train_df_encoded[class_column_name]
    # Define the groups (subject_id in your case)
    groups = train_df_encoded['patient_id']
    
    
    # Iterate through the splits
    for i, (train_index, val_idx) in enumerate(sgkf.split(X, y,groups)):
        print(f"Fold {i}:")
        # Ensure that the validation set is balanced
        unique_classes, class_counts = np.unique(y[val_idx], return_counts=True)
        print('unique_classes:', unique_classes)
        min_class_count = min(class_counts)

        balanced_val_indices = []
        for cls in unique_classes:
            cls_indices = val_idx[y[val_idx] == cls]
            balanced_val_indices.extend(cls_indices[:min_class_count])

        kfold_dict[f'val_{i}'].extend(balanced_val_indices)
        kfold_dict[f'train_{i}'].extend(train_index)

        kfold_dist_dict[f'val_{i}'] = dict(y[balanced_val_indices].value_counts())
        kfold_dist_dict[f'train_{i}'] = dict(y[train_index].value_counts())
        # If you want to see the class distribution in each fold
        print(f'Fold {len(train_index)} Class Distribution:')
        print(y[balanced_val_indices].value_counts())
    return kfold_dict, kfold_dist_dict

In [11]:
kfold_dict, kfold_dist_dict = create_StratifiedGroupKFold_splits(k, train_df,
                                        subject_id_column_name,
                                        class_column_name)

Fold 0:
unique_classes: [0 1 2 4]
Fold 33325 Class Distribution:
0    1125
1    1125
2    1125
4    1125
Name: label_flair, dtype: int64
Fold 1:
unique_classes: [0 1 2 4]
Fold 33325 Class Distribution:
0    1095
1    1095
2    1095
4    1095
Name: label_flair, dtype: int64
Fold 2:
unique_classes: [0 1 2 4]
Fold 33480 Class Distribution:
0    1141
1    1141
2    1141
4    1141
Name: label_flair, dtype: int64


In [12]:
kfold_dist_dict

{'train_0': {0: 15421, 1: 12005, 4: 3663, 2: 2236},
 'train_1': {0: 15493, 1: 11929, 4: 3637, 2: 2266},
 'train_2': {0: 15484, 1: 12132, 4: 3644, 2: 2220},
 'val_0': {0: 1125, 1: 1125, 2: 1125, 4: 1125},
 'val_1': {0: 1095, 1: 1095, 2: 1095, 4: 1095},
 'val_2': {0: 1141, 1: 1141, 2: 1141, 4: 1141}}

In [ ]:
len(kfold_dict['train_0']),len(kfold_dict['train_1']),len(kfold_dict['train_2'])

In [ ]:
# Initialize lists to store train and validation indices
train_indices = []
val_indices = []


# Iterate through the splits
for train_idx, val_idx in sgkf.split(X, y, groups=groups):
    # Ensure that the validation set is balanced
    unique_classes, class_counts = np.unique(y[val_idx], return_counts=True)
    print(class_counts)
    print(unique_classes)
    break
    min_class_count = min(class_counts)

    balanced_val_indices = []
    for cls in unique_classes:
        cls_indices = val_idx[y[val_idx] == cls]
        balanced_val_indices.extend(cls_indices[:min_class_count])

    train_indices.append(train_idx)
    val_indices.append(balanced_val_indices)

    # If you want to see the class distribution in each fold
    print(f'Fold {len(train_indices)} Class Distribution:')
    print(y[balanced_val_indices].value_counts())
#     print(balanced_val_indices)
    
#     train_fold_df = df[df[subject_id_column_name].isin(train_idx)].reset_index(drop=True)
#     val_fold_df = df[df[subject_id_column_name].isin(balanced_val_indices)].reset_index(drop=True)
    val_fold_df = df.loc[balanced_val_indices].reset_index(drop=True)
    train_fold_df = df.loc[train_idx].reset_index(drop=True)
    
    break








In [ ]:
len(train_fold_df['subject_id']), len(val_fold_df['subject_id'])
print(dict( y[balanced_val_indices].value_counts()))
print(dict( y[train_idx].value_counts()))

In [ ]:
train_fold_df[train_fold_df['subject_id']== 'BraTS20_Training_212']

In [ ]:
len(groups[balanced_val_indices])

In [ ]:
len(np.unique(groups[balanced_val_indices]))

In [ ]:
val_fold_df.groupby(['subject_id']).count()

In [ ]:
val_fold_df['label_flair'].value_counts()

In [ ]:
train_fold_df['label_flair'].value_counts()

In [ ]:
train_fold_df['label_flair'].value_counts()

In [ ]:
print(len(val_fold_df[val_fold_df['label_flair'] == 'LGG']['subject_id'].value_counts()))
val_fold_df[val_fold_df['label_flair'] == 'LGG']['subject_id'].value_counts()

In [ ]:
print(len(train_fold_df[train_fold_df['label_flair'] == 'LGG']['subject_id'].value_counts()))
train_fold_df[train_fold_df['label_flair'] == 'LGG']['subject_id'].value_counts()

In [ ]:
def idx_to_class(idx):
    
    if idx == 4:label='discard'
    if idx == 0:label='healthy'
    if idx == 1:label='HGG'
    if idx == 2:label='LGG'      
    return label

train_fold_df['label_flair'] = train_fold_df['label_flair'].apply(lambda x: idx_to_class(x))
val_fold_df['label_flair'] = val_fold_df['label_flair'].apply(lambda x: idx_to_class(x))

In [ ]:
train_fold_df = train_fold_df[train_fold_df.label_flair != 'discard']
val_fold_df = val_fold_df[val_fold_df.label_flair != 'discard']

In [ ]:
groups

In [ ]:
x = list(set(subject_id_class) - set(test_subjects))
len(x)

In [ ]:
samples_per_subject = max(df_meta.groupby(['subject_id'])['label_flair'].count())#155
df = df_meta.copy()

mystring = 'healthy'
discard = 'discard'
test_size = 0.3

minority_samples = min(class_dict.values())
# min_samples
min_test_samples = test_size* minority_samples
print(int(np.round(min_test_samples)))

minorty_class = [key for key in class_dict if class_dict[key] == min_test_samples]
minorty_class

for key in class_dict:
    if key!= mystring and key!= discard:
        print(key)
        df_class =  df[df[class_column_name] == key]
        subject_id_class =  np.unique(df_class[subject_id_column_name]).tolist()
        print(key, f':{len(subject_id_class)}')
        
        train_subjects, test_subjects = train_test_split(subject_id_class ,
                                            test_size = test_size, 
                                            random_state = 42, 
                                            shuffle = True) 
        training_subjects_list.extend(train_subjects)
        test_subjects_list.extend(test_subjects)

In [ ]:
training_subjects_list=[]
test_subjects_list =[]

# class_dict = get_class_label_dict(df, class_column)

for key in class_dict:
    if key != class_healthy_name:
        df_class =  df[df[class_column] == key]
        subject_id_class =  np.unique(df_class[subject_id_column_name]).tolist()
        train_subjects, test_subjects = train_test_split(subject_id_class ,
                                            test_size = test_size, 
                                            random_state = 42, 
                                            shuffle = True) 
        training_subjects_list.extend(train_subjects)
        test_subjects_list.extend(test_subjects)


In [ ]:
train_df = pd.read_csv('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/splits/train_df.csv', index_col=0)
train_df

In [ ]:
del train_df['Brats20ID']
del train_df['Age']
del train_df['Survival_days']
del train_df['Extent_of_Resection']

In [ ]:
train_df.isnull().values.any()

In [ ]:
test_df = pd.read_csv('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/splits/test_df.csv', index_col=0)
del test_df['Brats20ID']
del test_df['Age']
del test_df['Survival_days']
del test_df['Extent_of_Resection']
test_df

In [ ]:
print('HGG:', len(np.unique(test_df[test_df[class_column_name]=='HGG']['subject_id'])))
print('LGG:', len(np.unique(test_df[test_df[class_column_name]=='LGG']['subject_id'])))
print('healthy:', len(np.unique(test_df[test_df[class_column_name]=='healthy']['subject_id'])))

In [ ]:
test_df.isnull().values.any()

In [ ]:
test_df[test_df[class_column_name]=='healthy']

In [ ]:
test_df[test_df[class_column_name]=='HGG']

In [ ]:
test_df[test_df[class_column_name]=='LGG']

In [ ]:
meta_csv_path = '/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/meta_data_survival_mapping.csv'
class_column_name = 'label_flair'
subject_id_column_name = 'subject_id'
csv_save_path = Path('/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/splits')
test_size = 0.25
class_healthy_name ='healthy'

split = True  
# test_size = 0.25

In [ ]:
def get_csv_after_discard(csv_path):
    """Load meta csv. Filter samples to be discarded."""
    meta_data = pd.read_csv(csv_path)
    meta_data = meta_data[meta_data.label_flair != 'discard']
    meta_data = meta_data[meta_data.label_t1 != 'discard']
    meta_data = meta_data[meta_data.label_t1ce != 'discard']
    meta_data = meta_data[meta_data.label_t2 != 'discard']
    meta_data_df  = meta_data.reset_index(drop =True)
    return meta_data_df

df = get_csv_after_discard(meta_csv_path)
df

In [ ]:
len(df[df['label_flair']=='LGG']['subject_id'].value_counts())
print('HGG:', len(np.unique(df[df[class_column_name]=='HGG']['subject_id'])))
print('LGG:', len(np.unique(df[df[class_column_name]=='LGG']['subject_id'])))
print('healthy:', len(np.unique(df[df[class_column_name]=='healthy']['subject_id'])))

In [ ]:
def get_class_label_dict(df, class_column):
    class_dict = df[class_column].value_counts().to_dict()
    return class_dict

class_dict = get_class_label_dict(df, class_column_name)
class_dict

In [ ]:
df.groupby(['subject_id']).value_counts(['label_flair'])

# Create balanced Test set

In [ ]:
training_subjects_list=[]
test_subjects_list =[]

class_dict = get_class_label_dict(df, class_column)

for key in class_dict:
    if key != class_healthy_name:
        df_class =  df[df[class_column] == key]
        subject_id_class =  np.unique(df_class[subject_id_column_name]).tolist()
        train_subjects, test_subjects = train_test_split(subject_id_class ,
                                            test_size = test_size, 
                                            random_state = 42, 
                                            shuffle = True) 
        training_subjects_list.extend(train_subjects)
        test_subjects_list.extend(test_subjects)


In [ ]:
def class_to_idx(label):
    idx = 0
    if label == 'discard':idx =4
    if label == 'healthy':idx =0
    if label == 'HGG':idx =1
    if label=='LGG': idx = 2        
    return idx


def get_class_label_dict(df, class_column):
    class_dict = df[class_column].value_counts().to_dict()
    return class_dict


def get_csv_after_discard(csv_path):
    """Load meta csv. Filter samples to be discarded."""
    meta_data = pd.read_csv(csv_path)
    meta_data = meta_data[meta_data.label_flair != 'discard']
    meta_data = meta_data[meta_data.label_t1 != 'discard']
    meta_data = meta_data[meta_data.label_t1ce != 'discard']
    meta_data = meta_data[meta_data.label_t2 != 'discard']
    meta_data_df  = meta_data.reset_index(drop =True)
    return meta_data_df



def get_metadata_csv_label_encoded(meta_data_df):
    """Load meta csv. Filter samples to be discarded and label encoding."""
    meta_data = meta_data_df.copy()
    meta_data['label_flair'] = meta_data['label_flair'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1'] = meta_data['label_t1'].apply(lambda x: class_to_idx(x))
    meta_data['label_t1ce'] = meta_data['label_t1ce'].apply(lambda x: class_to_idx(x))
    meta_data['label_t2'] = meta_data['label_t2'].apply(lambda x: class_to_idx(x))
    meta_data_df  = meta_data.reset_index(drop =True)
    return meta_data_df

def generate_subject_id_splits(df, 
                             class_column, 
                             test_size,
                             subject_id_column_name,
                             class_healthy_name):
    training_subjects_list=[]
    test_subjects_list =[]
    class_dict = get_class_label_dict(df, class_column)
    for key in class_dict:
        if key != class_healthy_name:
            df_class =  df[df[class_column] == key]
            subject_id_class =  np.unique(df_class[subject_id_column_name]).tolist()
            train_subjects, test_subjects = train_test_split(subject_id_class ,
                                                test_size = test_size, 
                                                random_state = 42, 
                                                shuffle = True) 
            training_subjects_list.extend(train_subjects)
            test_subjects_list.extend(test_subjects)
    return training_subjects_list, test_subjects_list


def create_and_save_split(meta_csv_path , 
                         class_column,
                         test_size,
                         subject_id_column_name,
                         class_healthy_name,
                         csv_save_path):
#     df = pd.read_csv(meta_csv_path)
    df = get_csv_after_discard(meta_csv_path)
    training_subjects_list, test_subjects_list = generate_subject_id_splits(df,
                                                                          class_column, 
                                                                          test_size, 
                                                                          subject_id_column_name, 
                                                                          class_healthy_name)
    train_df = df[df[subject_id_column_name].isin(training_subjects_list)].reset_index(drop=True)
    test_df = df[df[subject_id_column_name].isin(test_subjects_list)].reset_index(drop=True)
    
    train_df.to_csv(csv_save_path / 'train_df.csv')
    test_df.to_csv(csv_save_path / 'test_df.csv')
    return train_df, test_df


def load_train_test_csv(csv_path):
    if  os.path.exists(csv_path  / 'train_df.csv'):
        train_df = pd.read_csv(csv_save_path / 'train_df.csv', index_col=0)
        test_df = pd.read_csv(csv_save_path / 'test_df.csv', index_col=0)          
    return train_df, test_df


def preprocess_train_df(train_df, subject_id_column_name):
    train_df = train_df.fillna(0)
    train_df = get_metadata_csv_label_encoded(train_df)
    train_df.set_index(subject_id_column_name, inplace=True, drop=False)
    return train_df


In [ ]:

if split:
    train_df, test_df = load_train_test_csv(csv_save_path)
else:
    train_df, test_df =  create_and_save_split(meta_csv_path , 
                                         class_column_name,
                                         test_size,
                                         subject_id_column_name,
                                         class_healthy_name,
                                         csv_save_path)


In [ ]:
#StratifiedGroupKFold
train_df_processed =  preprocess_train_df(train_df, subject_id_column_name)
train_df_processed

In [ ]:

np.unique(train_df_processed[subject_id_column_name].value_counts())

In [ ]:
np.unique(train_df[subject_id_column_name].value_counts())

In [ ]:
train_df

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
class_dict = get_class_label_dict(train_df, class_column_name)
print(class_dict)

In [ ]:
class_dict_test = get_class_label_dict(test_df, class_column_name)
print(class_dict_test)

In [ ]:
len(np.unique(train_df[subject_id_column_name]).tolist()), len(np.unique(test_df[subject_id_column_name]).tolist())

In [ ]:

training_subjects_list=[]
test_subjects_list =[]

training_subjects_list.extend(np.unique(train_df[subject_id_column_name]).tolist())
test_subjects_list.extend(np.unique(test_df[subject_id_column_name]).tolist())

result_split = any(item in test_subjects_list for item in training_subjects_list)
result_split

In [ ]:
def check_if_overlapping_split(train_df, test_df):
    
    training_subjects_list=[]
    test_subjects_list =[]

    training_subjects_list.extend(np.unique(train_df[subject_id_column_name]).tolist())
    test_subjects_list.extend(np.unique(test_df[subject_id_column_name]).tolist())

    result_split = any(item in test_subjects_list for item in training_subjects_list)
    print(result_split)
    return 

# Load train_df and meta_csv to split into  KFold

In [ ]:
def prepare_train_data_split(meta_csv_path, train_df, subject_id_column_name):
    #read meta_data_csv with 'discard' as well
    df_meta = pd.read_csv(meta_csv_path)
    #get the correct indexes (subject_id) from split train_df(as train & test splits also accounts the class distribution)
    # use the only indexes from train_df to be considered from meta_data_csv
    training_subjects_list =  np.unique(train_df[subject_id_column_name]).tolist()

    train_df_indexed = df_meta[df_meta[subject_id_column_name].isin(training_subjects_list)].reset_index(drop=True)
    return train_df_indexed

In [ ]:
train_df_indexed = prepare_train_data_split(meta_csv_path, train_df, subject_id_column_name)
train_df_indexed

In [ ]:
np.unique(train_df_indexed[subject_id_column_name].value_counts())

# GroupKFold

In [ ]:
from sklearn.model_selection import GroupKFold,  StratifiedKFold, StratifiedGroupKFold

In [ ]:


def create_groupKFold_splits(df_indexed,class_column_name, subject_id_column_name,  k =3):
    
    kfold_dict = {f"train_{i}": [] for i in range(k)}
    kfold_dict.update({f"val_{i}": [] for i in range(k)})
    
    df = df_indexed.copy()

    X = df
    y = df[class_column_name]
    
    groups = df[subject_id_column_name].values
    group_kfold = GroupKFold(n_splits=3)
    group_kfold.get_n_splits(X, y, groups)
    
    for i, (train_index, val_index) in enumerate(group_kfold.split(X, y,groups)):        
        print(f"Fold {i}:")
        kfold_dict[f'train_{i}'].extend(groups[train_index])
        kfold_dict[f'val_{i}'].extend(groups[val_index])
        
    return kfold_dict

In [ ]:
k=3
kfold_dict =  create_groupKFold_splits(train_df_indexed,class_column_name, subject_id_column_name,  k)
kfold_dict

In [ ]:
kfold_dict.keys()

In [ ]:
train_df_indexed.set_index(subject_id_column_name, inplace=True, drop=False)

In [ ]:
train_df_indexed

In [ ]:
def get_df_using_kfold_indexes(df_indexed, class_column_name, subject_id_column_name, train_index, val_index):
    
    df = df_indexed.copy()
#     train_df = df.iloc[train_index]
#     val_df = df.iloc[val_index]
    
    train_df = df[df[subject_id_column_name].isin(train_index)].reset_index(drop=True)
    val_df = df[df[subject_id_column_name].isin(val_index)].reset_index(drop=True)
    
    train_df[train_df[class_column_name]!='discard']
    val_df[val_df[class_column_name]!='discard']
    return train_df, val_df

i =0
if i<k:
    for key in kfold_dict:        
        train_index =  kfold_dict[f'train_{i}']
        print(train_index)
        val_index = kfold_dict[f'val_{i}']

        train_df, val_df = get_df_using_kfold_indexes(train_df_indexed, class_column_name, 'subject_id', train_index, val_index)
        break
        train_df
        # dataloader loop and continue to trainig
    i=+1

In [ ]:
train_df

In [ ]:
kfold_dict.keys()

In [ ]:
i =0
train_index =  kfold_dict[f'train_{i}']
val_index = kfold_dict[f'val_{i}']

# print('train_index:', train_index)
# print('val_index:', val_index)

In [ ]:
len(np.unique(train_index)), len(np.unique(val_index))

In [ ]:
#Initiaze the model outside traini method:
# model = get_model_by_name(model_name,num_classes, mod, device_index, to_use_pre_trained_weights )
# device_index = f'cuda:{device_num}'


def get_df_using_kfold_indexes(df_indexed, class_column_name, train_index, val_index):
    
    df = df_indexed.copy()
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]
    
    train_df[train_df[class_column_name]!='discard']
    val_df[val_df[class_column_name]!='discard']
    return train_df, val_df

# def train_split_df_kfold_dict(df_indexed,class_column_name, kfold_dict, k):
#     i =0
#     if i<k:
#         for key,value in kfold_dict:        
#             train_index =  kfold_dict[f'train_{i}']
#             val_index = kfold_dict[f'val_{i}']

#             train_df, val_df = get_df_using_kfold_indexes(df_indexed, class_column_name, train_index, val_index)
#             # dataloader loop and continue to trainig
#         i=+1


        
def train_split_df_kfold_dict(df_indexed, class_column_name, kfold_dict, k, test_df):
    fold =0
    if fold < k:        
        train_index =  kfold_dict[f'train_{i}']
        val_index = kfold_dict[f'val_{i}']
        train_df, val_df = get_df_using_kfold_indexes(df_indexed, 
                                                      class_column_name, 
                                                      train_index,
                                                      val_index)
        # dataloader loop and continue to trainig
        if mod == 'flair_t1ce_t2':
            train_dataloader, val_dataloader, test_dataloader =  get_multimodal_dataloader(train_df,
                                                                                           test_df, 
                                                                                           val_df,
                                                                                           batch_size)
        else:
            train_dataloader, val_dataloader, test_dataloader =  get_dataloader(train_df,
                                                                                test_df,
                                                                                val_df,
                                                                                mod,
                                                                                batch_size)
        trained_model = start_train(model.to(device_index), train_dataloader, 
                                    val_dataloader,test_dataloader,
                                    n_epochs , output_dir, 
                                    device_index, model_name, fold)
        fold=+1
    
    

In [ ]:
test_df['label_flair'].value_counts()

In [ ]:
def plot_training_cruves(train_acc, val_acc, train_loss, val_loss, train_f1_scores, val_f1_scores, saving_dir, fold):    
    fig = plt.figure(figsize=(10,5))
    plt.title(f"Train-Validation Accuracy_fold{fold}")
    plt.plot(train_acc, label='train')
    plt.plot(val_acc, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('accuracy', fontsize=12)
    plt.legend(loc='best')
    plt.savefig(saving_dir / f'fold{fold}' / f'training_acc_curve_fold{fold}.png',  bbox_inches='tight')
    save_list_to_csv(train_acc, f'list_train_acc_fold{fold}', saving_dir / f'fold{fold}')
    save_list_to_csv(val_acc, f'list_val_acc_fold{fold}', saving_dir /  f'fold{fold}')

    fig = plt.figure(figsize=(10,5))
    plt.title(f"Train-Validation F1 score_fold{fold}")
    plt.plot(train_f1_scores, label='train')
    plt.plot(val_f1_scores, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('f1 score', fontsize=12)
    plt.legend(loc='best')
    plt.savefig(saving_dir / f'fold{fold}' / f'training_f1_curve_fold{fold}'.png',  bbox_inches='tight')
    save_list_to_csv(train_f1_scores, f'list_train_f1_fold{fold}', saving_dir / f'fold{fold}')
    save_list_to_csv(val_f1_scores, f'list_val_f1_fold{fold}', saving_dir / f'fold{fold}')

    fig = plt.figure(figsize=(10,5))
    plt.title(f"Train-Validation Loss_fold{fold}")
    plt.plot(train_loss, label='train')
    plt.plot(val_loss, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('loss', fontsize=12)
    plt.legend(loc='best')
    plt.savefig(saving_dir / f'fold{fold}' / f'training_loss_curve_fold{fold}.png',  bbox_inches='tight')
    save_list_to_csv(train_loss, f'list_train_loss_fold{fold}', saving_dir / f'fold{fold}')
    save_list_to_csv(val_loss, f'list_val_loss_fold{fold}', saving_dir / f'fold{fold}')

In [ ]:
def get_confusion_matrix(model, dataloader, saving_dir, device_index, fold):    
    device = torch.device(device_index) if torch.cuda.is_available() else torch.device('cpu') 
    np.random.seed(31101995)
    torch.manual_seed(31101995)
    labels = ['healthy', 'HGG','LGG']
    label_dict ={'0':'healthy', '1':'HGG','2':'LGG'}
    y_pred_list = []
    y_true_list = []    
    with torch.no_grad():
        model.eval()
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            _, y_pred_tag = torch.max(y_test_pred, dim = 1)
                        
            y_pred_list.append(y_pred_tag.cpu().numpy())
            y_true_list.append(y_batch.cpu().numpy())      
    y_pred_list = [i[0] for i in y_pred_list]
    y_true_list = [i[0] for i in y_true_list]

    print(classification_report(y_true_list, y_pred_list))
    classification_report_df = pd.DataFrame(classification_report(y_true_list, y_pred_list, output_dict=True))
    classification_report_df.rename(columns=label_dict, inplace=True)
    cm = confusion_matrix(y_true_list, y_pred_list)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    # Add numbers to the confusion matrix heatmap.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x=j, y=i, s=cm[i, j], va='center', ha='center', color='white')
    plt.title('Confusion matrix of the classifier')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(saving_dir / f'fold{fold}' / f'confusion_matrix_fold{fold}.png')
    plt.show()
    confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list),index=labels)
    confusion_matrix_df.to_csv( saving_dir / f'fold{fold}' / f'confusion_matrix_df_fold{fold}.csv',index=False )
    classification_report_df.to_csv( saving_dir / f'fold{fold}' / f'classification_report_df_fold{fold}.csv',index=False )
    return confusion_matrix_df, classification_report_df

In [ ]:
fold0_csv = '/home/shsingh/knowledge_distillation/dataset/scratch/dataframes/splits/fold_0/train_df_fold_0.csv'
fold0_df = pd.read_csv(fold0_csv)
fold0_df

In [ ]:
len(fold0_df['subject_id'].values)

In [ ]:
fold0_df[fold0_df['subject_id']== 'BraTS20_Training_001']

In [ ]:
kfold_eval_path ='/home/shsingh/knowledge_distillation/kfold_result/flair/baseline/densenet121/fold0/list_train_acc_fold0.csv'
kfold_eval_train_acc  = pd.read_csv(kfold_eval_path)
# kfold_eval_train_acc = kfold_eval_train_acc.T.reset_index()
# kfold_eval_train_acc = kfold_eval_train_acc.rename(columns={"index": "train_acc"})

# kfold_eval_train_acc['train_acc'] = kfold_eval_train_acc['train_acc'].round(3)
kfold_eval_train_acc

In [ ]:
df_acc = kfold_eval_train_acc.copy()
df = pd.DataFrame(df_acc.T)
df.reset_index(level=0, inplace=True)
# Rename the column
df = df.rename(columns={'index': 'accuracy'})
# Convert the 'accuracy' column to numeric type
df['accuracy'] = df['accuracy'].astype(str).str.replace(r'(\d+\.\d+)\.(\d+)', r'\1\2')
df['accuracy'] = pd.to_numeric(df['accuracy'], errors='coerce')
# Round the 'accuracy' column to 4 decimal places
df['accuracy'] = df['accuracy'].round(4)
df = df.rename_axis('epoch')
df

In [ ]:
plt.plot( df['accuracy'])
# Set plot title and labels
plt.title('Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
kfold_eval_path_val ='/home/shsingh/knowledge_distillation/kfold_result/flair/baseline/densenet121/fold0/list_val_acc_fold0.csv'
kfold_eval_val_acc  = pd.read_csv(kfold_eval_path_val)

df_val = kfold_eval_val_acc.copy()
dfff = pd.DataFrame(df_val.T)
dfff.reset_index(level=0, inplace=True)
# Rename the column
dfff = dfff.rename(columns={'index': 'accuracy'})
# Convert the 'accuracy' column to numeric type
dfff['accuracy'] = dfff['accuracy'].astype(str).str.replace(r'(\d+\.\d+)\.(\d+)', r'\1\2')
dfff['accuracy'] = pd.to_numeric(dfff['accuracy'], errors='coerce')
# Round the 'accuracy' column to 4 decimal places
dfff['accuracy'] = dfff['accuracy'].round(4)
dfff = dfff.rename_axis('epoch')


plt.plot( dfff['accuracy'])
# Set plot title and labels
plt.title('Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
fold0_loss_path ='/home/shsingh/knowledge_distillation/kfold_result/flair/baseline/efficientnet_v2_s/fold0/list_train_loss_fold0.csv'
fold0_loss_df  = pd.read_csv(fold0_loss_path)

# df_val = fold0_loss_df.copy()
fold0_loss_df = pd.DataFrame(fold0_loss_df.T)
fold0_loss_df.reset_index(level=0, inplace=True)
# Rename the column
fold0_loss_df = fold0_loss_df.rename(columns={'index': 'loss'})
# Convert the 'accuracy' column to numeric type
fold0_loss_df['loss'] = fold0_loss_df['loss'].astype(str).str.replace(r'(\d+\.\d+)\.(\d+)', r'\1\2')
fold0_loss_df['loss'] = pd.to_numeric(fold0_loss_df['loss'], errors='coerce')
# Round the 'accuracy' column to 4 decimal places
fold0_loss_df['loss'] = fold0_loss_df['loss'].round(4)
fold0_loss_df = fold0_loss_df.rename_axis('epoch')


plt.plot( fold0_loss_df['loss'])
# Set plot title and labels
plt.title('loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('loss')

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
kfold_result = Path('/home/shsingh/knowledge_distillation/kfold_result')
mod = 'flair'
model_type = 'baseline'
model_name = 'densenet121'
fold = 'fold0'
metric = 'acc' #f1, loss

In [ ]:
def get_metric_plotting_df(df_metric, metric):    
    df = pd.DataFrame(df_metric.T)
    df.reset_index(level=0, inplace=True)
    
    metric_col_name =  metric
        
    # Rename the column
    df = df.rename(columns={'index': metric_col_name})
    # Convert the 'accuracy' column to numeric type
    df[metric_col_name] = df[metric_col_name].astype(str).str.replace(r'(\d+\.\d+)\.(\d+)', r'\1\2')
    df[metric_col_name] = pd.to_numeric(df[metric_col_name], errors='coerce')
    # Round the 'accuracy' column to 4 decimal places
    df[metric_col_name] = df[metric_col_name].round(4)
        
    df = df.rename_axis('epoch')
    return df

In [ ]:
def get_and_save_trainig_plots(mod, model_type, model_name, fold, metric ):
    
    kfold_result = Path('/home/shsingh/knowledge_distillation/kfold_result')
    fold_path =  kfold_result/ mod / model_type / model_name / fold 
    
    train_csv_path =  fold_path / f'list_train_{metric}_{fold}.csv'
    val_csv_path = fold_path / f'list_val_{metric}_{fold}.csv'
    
    train_df = pd.read_csv(train_csv_path)
    val_df = pd.read_csv(val_csv_path)

    
    if metric == 'acc':   
        metric = 'accuracy'
    else:
        metric = metric
        
    
    
    train_df_plot = get_metric_plotting_df(train_df, metric)
    val_df_plot = get_metric_plotting_df(val_df, metric)
    
    train_df_plot[metric] = train_df_plot[metric] / 10
#     val_df_plot[metric] = val_df_plot[metric] / 100
    
    plt.plot(train_df_plot[metric], label='train')
    plt.plot(val_df_plot[metric], label='validation')

    # Set plot title and labels
    plt.title(f'{metric} vs Epoch')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    # Add legend
    plt.legend(['train', 'val'],loc='best')
    # Show the plot
    plt.show()
    return  train_df_plot, val_df_plot  

In [ ]:

mod = 'flair'
model_type = 'baseline'
model_name = 'densenet121'
fold = 'fold0'
metric = 'loss' #f1, loss

train_df_plot, val_df_plot   = get_and_save_trainig_plots(mod, model_type, model_name, fold, metric )

In [ ]:
train_df_plot

# DRAFT code

In [ ]:
def create_groupKFold_splits(df,class_column_name, subject_id_column_name,  k =3):
    
    train_0 = []
    train_1 = []
    train_2 = []

    val_0 = []
    val_1 = []
    val_2 = []
    
    X = df
    y = df[class_column_name]
    
    groups = df[subject_id_column_name].values
    group_kfold = GroupKFold(n_splits=3)
    group_kfold.get_n_splits(X, y, groups)
    
    for i, (train_index, test_index) in enumerate(group_kfold.split(X, y,groups)):
        print(f"Fold {i}:")
        if i == 0:
            train_0.extend(groups[train_index])
            val_0.extend(groups[test_index])
        if i == 1:
            train_1.extend(groups[train_index])
            val_1.extend(groups[test_index])
        if i == 2:
            train_2.extend(groups[train_index])
            val_2.extend(groups[test_index])

        print(f"  Train: index={train_index}, group={groups[train_index]}")
        print(f"  Test:  index={test_index}, group={groups[test_index]}")
        
    print(any(item in np.unique(val_0) for item in np.unique(train_0)))
    print(any(item in np.unique(val_1) for item in np.unique(train_1)))
    print(any(item in np.unique(val_2) for item in np.unique(train_2)))
    return  train_0, val_0, train_1, val_1, train_2, val_2

In [ ]:
train_0, val_0, train_1, val_1, train_2, val_2 = create_groupKFold_splits(train_df_indexed,
                                                                          class_column_name,
                                                                          subject_id_column_name,
                                                                          k =3)

In [ ]:
print(len(np.unique(train_0)), len(np.unique(val_0)) )

any(item in np.unique(val_0) for item in np.unique(train_0))

# np.unique(train_0.tolist()),np.unique(train_0.tolist())

In [ ]:
train_df_indexed[train_df_indexed['label_flair']!='discard']

In [ ]:
k=3
for i in range(k):
    print(i)

In [ ]:

kfold_dict

In [ ]:
kfold_dict = dict.fromkeys(fold_ids, [])
kfold_dict

In [ ]:
kfold_dict = {f"train_{i}": [] for i in range(k)}
kfold_dict.update({f"val_{i}": [] for i in range(k)})


df = train_df_indexed.copy()

X = df
y = df[class_column_name]

groups = df[subject_id_column_name].values
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(X, y, groups)

for i, (train_index, val_index) in enumerate(group_kfold.split(X, y,groups)):
    print(f"Fold {i}:")
    kfold_dict[f'train_{i}'].extend(groups[train_index])
    kfold_dict[f'val_{i}'].extend(groups[val_index])
#     if i == 0:
#         train_0.extend(groups[train_index])
#         val_0.extend(groups[test_index])
#     if i == 1:
#         train_1.extend(groups[train_index])
#         val_1.extend(groups[test_index])
#     if i == 2:
#         train_2.extend(groups[train_index])
#         val_2.extend(groups[test_index])

    print(f"  Train: index={train_index}, group={groups[train_index]}")
    print(f"  Test:  index={val_index}, group={groups[val_index]}")

In [ ]:
kfold_dict
for key, value in kfold_dict.items():
    unique_values = set(value)
    print(f"Unique elements in {key}: {unique_values}")
    print('XXXXXXXXXXXXXXXXXXXXX')

In [ ]:

group_0 = []
group_1 = []
group_2 = []

group_0_val = []
group_1_val = []
group_2_val = []

X = df
y = df[class_column_name]
groups = df['subject_id'].values
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(X, y, groups)
# print(groups)
# print(group_kfold)
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y,groups)):
    print(f"Fold {i}:")
    if i == 0:
        group_0.extend(groups[train_index])
        group_0_val.extend(groups[test_index])
    if i == 1:
        group_1.extend(groups[train_index])
        group_1_val.extend(groups[test_index])
    if i == 2:
        group_2.extend(groups[train_index])
        group_2_val.extend(groups[test_index])

    print(f"  Train: index={train_index}, group={groups[train_index]}")
#     print('train_group:',np.unique() )
    print(f"  Test:  index={test_index}, group={groups[test_index]}")
#     train_data = df.iloc[train_index]
#     val_data = df.iloc[test_index]
    
    

In [ ]:
def check_if_overlappin_splits():
    print(any(item in np.unique(group_0_val) for item in np.unique(group_0)))
    print(any(item in np.unique(group_1_val) for item in np.unique(group_1)))
    print(any(item in np.unique(group_2_val) for item in np.unique(group_2))
        return

In [ ]:
train_lis = np.unique(train_data['subject_id'])
test_lis = np.unique(val_data['subject_id'])

any(item in test_lis for item in train_lis)


In [ ]:

print(any(item in np.unique(group_0_val) for item in np.unique(group_0)))
print(any(item in np.unique(group_1_val) for item in np.unique(group_1)))
print(any(item in np.unique(group_2_val) for item in np.unique(group_2)))

In [ ]:

print(any(item in np.unique(group_0_val) for item in np.unique(group_1_val)))
print(any(item in np.unique(group_1_val) for item in np.unique(group_2_val)))
print(any(item in np.unique(group_2_val) for item in np.unique(group_0_val)))

In [ ]:

print(any(item in np.unique(group_0_val) for item in np.unique(group_1_val)))
print(any(item in np.unique(group_1_val) for item in np.unique(group_2_val)))
print(any(item in np.unique(group_2_val) for item in np.unique(group_0_val)))